In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("../input/us-baby-names/NationalNames.csv")

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data['Name'].nunique()

In [ ]:
data['Gender'] = data['Gender'].astype('category').cat.codes

In [ ]:
df = data.groupby('Name').mean()['Gender'].reset_index()
df['Gender'] = df['Gender'].astype('int')
df.head()

In [ ]:
import string

In [ ]:
letters = list(string.ascii_lowercase)
letters

In [ ]:
vocab = dict(zip(letters,range(1,27)))
vocab

In [ ]:
r_vocab = dict(zip(range(1,27),letters))
r_vocab

In [ ]:
def word_to_number():
  for i in range(0,df.shape[0]):
    seq = [ vocab[letters.lower()] for letters in df['Name'][i] ]
    df['Name'][i] = seq

In [ ]:
word_to_number()

In [ ]:
df.head()

In [ ]:
x = df['Name'].values
y = df['Gender'].values

In [ ]:
name_length = [ len(x[i]) for i in range(0,df.shape[0]) ]

In [ ]:
name_length

In [ ]:
plt.hist(name_length,bins=20)
plt.show()

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
X = pad_sequences(df['Name'].values,maxlen=10,padding='pre')
X

In [ ]:
from keras.layers import Input,Embedding,LSTM,Dense
from keras.models import Model

In [ ]:
vocab_size = len(vocab)+1

In [ ]:
inp = Input(shape=(10,))
emn = Embedding(input_dim=vocab_size, output_dim=5)(inp)

lstm1 = LSTM(units=32, return_sequences=True)(emn)
lstm2 = LSTM(units=64)(lstm1)

out = Dense(units=1, activation='sigmoid')(lstm2)
my_model = Model(inputs=inp, outputs=out)

In [ ]:
my_model.summary()

In [ ]:
my_model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['acc'])

In [ ]:
his = my_model.fit(X,y,epochs=10,batch_size=256,validation_split=0.2)

In [ ]:
h = his.history

In [ ]:
plt.plot(h['acc'])
plt.plot(h['val_acc'])
plt.show()

In [ ]:
name = "Marry"
test_name = name.lower()

seq = [vocab[i] for i in test_name]
x_test = pad_sequences([seq],10)
x_test

In [ ]:
y_pred = my_model.predict(x_test)
y_pred

In [ ]:
if y_pred<0.5:
  print('Female')
else:
  print('Male')